Antes de nada: 
pip install xgboost shap scikit-learn pandas matplotlib


In [7]:
import pandas as pd
import xgboost as xgb
import pickle
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import shap

In [8]:
features_df = pd.read_csv("features_per_patient_avg.csv")
print(features_df.columns)


Index(['I_r_amp', 'I_s_amp', 'I_r_s_ratio', 'I_qrs_dur', 'I_t_polarity',
       'II_r_amp', 'II_s_amp', 'II_r_s_ratio', 'II_qrs_dur', 'II_t_polarity',
       'III_r_amp', 'III_s_amp', 'III_r_s_ratio', 'III_qrs_dur',
       'III_t_polarity', 'AVR_r_amp', 'AVR_s_amp', 'AVR_r_s_ratio',
       'AVR_qrs_dur', 'AVR_t_polarity', 'AVL_r_amp', 'AVL_s_amp',
       'AVL_r_s_ratio', 'AVL_qrs_dur', 'AVL_t_polarity', 'AVF_r_amp',
       'AVF_s_amp', 'AVF_r_s_ratio', 'AVF_qrs_dur', 'AVF_t_polarity',
       'V1_r_amp', 'V1_s_amp', 'V1_r_s_ratio', 'V1_qrs_dur', 'V1_t_polarity',
       'V2_r_amp', 'V2_s_amp', 'V2_r_s_ratio', 'V2_qrs_dur', 'V2_t_polarity',
       'V3_r_amp', 'V3_s_amp', 'V3_r_s_ratio', 'V3_qrs_dur', 'V3_t_polarity',
       'V4_r_amp', 'V4_s_amp', 'V4_r_s_ratio', 'V4_qrs_dur', 'V4_t_polarity',
       'V5_r_amp', 'V5_s_amp', 'V5_r_s_ratio', 'V5_qrs_dur', 'V5_t_polarity',
       'V6_r_amp', 'V6_s_amp', 'V6_r_s_ratio', 'V6_qrs_dur', 'V6_t_polarity',
       'PatientID'],
      dtype='object')

We want to load our characteristic data from features_per_patient and full_data_corrected_2024 to unite them correcly and then train a XGBoost model. 

In [9]:
with open("data/full_data_corrected_2024.pkl", "rb") as f:
    full_data = pickle.load(f)

In [10]:
print(type(full_data))  # ¿Es dict? ¿DataFrame?
if isinstance(full_data, dict):
    print(full_data.keys())  # Ver las claves
elif isinstance(full_data, pd.DataFrame):
    print(full_data.columns)  # Ver columnas


<class 'dict'>
dict_keys(['I', 'II', 'III', 'AVR', 'AVL', 'AVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'Sex', 'HTA', 'Age', 'PVC_transition', 'SOO_chamber', 'Height', 'Weight', 'BMI', 'DM', 'DLP', 'Smoker', 'COPD', 'Sleep_apnea', 'CLINICAL_SCORE', 'SOO', 'OTorigin'])


In [15]:
import pandas as pd
df_labels = pd.DataFrame(full_data)
print(df_labels.columns)
df_labels['PatientID'] = features_df['PatientID']
data = pd.merge(features_df, df_labels[['PatientID', 'SOO']], on='PatientID')

X = data.drop(columns=['PatientID', 'SOO'])
y = data['SOO']

print(X.shape, y.shape)
print(y.unique())

Index(['I', 'II', 'III', 'AVR', 'AVL', 'AVF', 'V1', 'V2', 'V3', 'V4', 'V5',
       'V6', 'Sex', 'HTA', 'Age', 'PVC_transition', 'SOO_chamber', 'Height',
       'Weight', 'BMI', 'DM', 'DLP', 'Smoker', 'COPD', 'Sleep_apnea',
       'CLINICAL_SCORE', 'SOO', 'OTorigin'],
      dtype='object')
(181, 60) (181,)
['Unión mitroaórtica' 'Pared libre anterior' 'RCC'
 'M. papilar posteromedial' 'RVOT septal' 'Seno de Valsalva izdo.'
 'Intersenos dcho.-izdo.' 'LVOT intersenos dcho.-izdo.'
 'RVOT posterolateral' 'RVOT' 'Inferoseptal basal'
 'Seno de Valsalva dcho.' 'RVOT posterior' 'Septo basal izquierdo'
 'Ostium del SC (crux cordis)' 'Anillo tricuspídeo inferior/inferoseptal'
 'Infundíbulo' 'RVOT anterolateral' 'Summit' 'Fascicular posterior'
 'CS distal (summit)' 'RVOT medioseptal' 'septo' 'RVOT lateral + septal'
 'RVOT anteroseptal' 'Anillo tricuspídeo lateral' 'Intersenos' 'LV summit'
 'M. papilar anterolateral' 'LV summit EPI (SC)' 'Banda moderadora'
 'VI anteroseptal basal' nan 'LVOT anterola